### 3bead RNA system

In [1]:
import openmm as mm
from openmm.app import *
from openmm import *
from openmm.unit import *
from openmm import unit
import open3SPN2
import pandas 
import numpy as np
import openmm.unit as u

In [85]:
def writePDB(atoms,pdb_file):
    with open(pdb_file, 'w+') as pdb:
        for i, atom in atoms.iterrows():
            pdb_line = f'{atom.recname:<6}{atom.serial:>5} {atom["name"]:^4}{atom.altLoc:1}'+\
                       f'{atom.resname:<3} {atom.chainID:1}{atom.resSeq:>4}{atom.iCode:1}   '+\
                       f'{atom.x:>8.3f}{atom.y:>8.3f}{atom.z:>8.3f}' +\
                       f'{atom.occupancy:>6.2f}{atom.occupancy:>6.2f}'+' ' * 10 +\
                       f'{atom.element:>2}{atom.charge:>2}'
            assert len(pdb_line) == 80, f'An item in the atom table is longer than expected ({len(pdb_line)})\n{pdb_line}'
            pdb.write(pdb_line + '\n')
            
            
def pdb2table(pdb):
    """ Parses a pdb in the openmm format and outputs a table that contains all the information
    on a pdb file """
    cols = ['recname', 'serial', 'name', 'altLoc',
            'resname', 'chainID', 'resSeq', 'iCode',
            'x', 'y', 'z', 'occupancy', 'tempFactor',
            'element', 'charge']
    data = []
    for atom, pos in zip(pdb.topology.atoms(), pdb.positions):
        residue = atom.residue
        chain = residue.chain
        pos = pos.value_in_unit(unit.angstrom)
        data += [dict(zip(cols, ['ATOM', int(atom.id), atom.name, '',
                                 residue.name, chain.id, int(residue.id), '',
                                 pos[0], pos[1], pos[2], 0, 0,
                                 atom.element.symbol, '']))]
    atom_list = pandas.DataFrame(data)
    atom_list = atom_list[cols]
    atom_list.index = atom_list['serial']
    return atom_list


def CoarseGrain2(pdb_table, rna=True):
        """ Selects DNA atoms from a pdb table and returns a table containing only the coarse-grained atoms for 3SPN2"""
        masses = {"H": 1.00794, "C": 12.0107, "N": 14.0067, "O": 15.9994, "P": 30.973762, }
        CG = {"O5\'": 'P', "C5\'": 'S', "C4\'": 'S', "O4\'": 'S', "C3\'": 'S', "O3\'": 'P',
              "C2\'": 'S', "C1\'": 'S', "O5*": 'P', "C5*": 'S', "C4*": 'S', "O4*": 'S',
              "C3*": 'S', "O3*": 'P', "C2*": 'S', "C1*": 'S', "N1": 'B', "C2": 'B', "O2": 'B',
              "N2": 'B', "N3": 'B', "C4": 'B', "N4": 'B', "C5": 'B', "C6": 'B', "N9": 'B',
              "C8": 'B', "O6": 'B', "N7": 'B', "N6": 'B', "O4": 'B', "C7": 'B', "P": 'P',
              "OP1": 'P', "OP2": 'P', "O1P": 'P', "O2P": 'P', "OP3": 'P', "HO5'": 'P',
              "H5'": 'S', "H5''": 'S', "H4'": 'S', "H3'": 'S', "H2'": 'S', "H2''": 'S',
              "H1'": 'S', "H8": 'B', "H61": 'B', "H62": 'B', 'H2': 'B', 'H1': 'B', 'H21': 'B',
              'H22': 'B', 'H3': 'B', 'H71': 'B', 'H72': 'B', 'H73': 'B', 'H6': 'B', 'H41': 'B',
              'H42': 'B', 'H5': 'B', "HO3'": 'P'}
        if rna:
            CG.update({"HO2\'": 'S', "O2\'": 'S'})
        cols = ['recname', 'serial', 'name', 'altLoc',
                'resname', 'chainID', 'resSeq', 'iCode',
                'x', 'y', 'z', 'occupancy', 'tempFactor',
                'element', 'charge', 'type']
        temp = pdb_table.copy()

        # Select DNA residues
        if rna:
            temp = temp[temp['resname'].isin(['A', 'U', 'G', 'C'])]
        else:
            temp = temp[temp['resname'].isin(['DA', 'DT', 'DG', 'DC'])]

        # Group the atoms by sugar, phosphate or base
        temp['group'] = temp['name'].replace(CG)
        temp = temp[temp['group'].isin(['P', 'S', 'B'])]

        # Move the O3' to the next residue
        for c in temp['chainID'].unique():
            sel = temp.loc[(temp['name'] == "O3\'") & (temp['chainID'] == c), "resSeq"]
            temp.loc[(temp['name'] == "O3\'") & (temp['chainID'] == c), "resSeq"] = list(sel)[1:] + [-1]
            sel = temp.loc[(temp['name'] == "O3\'") & (temp['chainID'] == c), "resname"]
            temp.loc[(temp['name'] == "O3\'") & (temp['chainID'] == c), "resname"] = list(sel)[1:] + ["remove"]
        #temp = temp[temp['resSeq'] > 0]
        temp = temp[temp['resname'] != 'remove']

        # Calculate center of mass
        temp['element'] = temp['element'].str.strip()
        temp['mass'] = temp.element.replace(masses).astype(float)
        temp[['x', 'y', 'z']] = (temp[['x', 'y', 'z']].T * temp['mass']).T[['x', 'y', 'z']]
        temp = temp[temp['element'] != 'H']  # Exclude hydrogens
        Coarse = temp.groupby(['chainID', 'resSeq', 'resname', 'group']).sum().reset_index()
        Coarse[['x', 'y', 'z']] = (Coarse[['x', 'y', 'z']].T / Coarse['mass']).T[['x', 'y', 'z']]

        # Set pdb columns
        Coarse['recname'] = 'ATOM'
        Coarse['name'] = Coarse['group']
        Coarse['altLoc'] = ''
        Coarse['iCode'] = ''
        Coarse['charge'] = ''
        # Change name of base to real base
        mask = (Coarse.name == 'B')
        Coarse.loc[mask, 'name'] = Coarse[mask].resname.str[-1]  # takes last letter from the residue name
        Coarse['type'] = Coarse['name']
        # Set element (depends on base)
        if rna:
            Coarse['name'] = Coarse['name'].replace(to_replace={'S'}, value='Sr')
        Coarse['element'] = Coarse['name'].replace({'P': 'P', 'S': 'H', 'Sr': 'D',
                                                    'A': 'N', 'T': 'S', 'U': 'S', 'G': 'C', 'C': 'O'})
        # Remove P from the beginning
        drop_list = []
        for chain in Coarse.chainID.unique():
            sel = Coarse[Coarse.chainID == chain]
            drop_list += list(sel[(sel.resSeq == sel.resSeq.min()) & sel['name'].isin(['P'])].index)
        Coarse = Coarse.drop(drop_list)

        # Renumber
        Coarse.index = range(len(Coarse))
        Coarse['serial'] = Coarse.index
        return Coarse[cols]

In [86]:
import MDAnalysis as mda
import nglview as nv
u = mda.Universe('GC_clean.pdb')
view = nv.show_mdanalysis(u)
view

/opt/homebrew/anaconda3/envs/openmm_all/lib/python3.9/site-packages/MDAnalysis/topology/PDBParser.py:348: UserWarning: Unknown element D found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/opt/homebrew/anaconda3/envs/openmm_all/lib/python3.9/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


NGLWidget()

In [87]:
fix=open3SPN2.fixPDB("G5C5.pdb")

In [88]:
all_a=pdb2table(fix)
all_a

,recname,serial,name,altLoc,resname,chainID,resSeq,iCode,x,y,z,occupancy,tempFactor,element,charge
serial,,,,,,,,,,,,,,,
1,ATOM,1,P,,G,A,1,,-5.943000,9.439000,-4.688000,0,0,P,
2,ATOM,2,OP1,,G,A,1,,-4.770000,10.265000,-4.323000,0,0,O,
3,ATOM,3,OP2,,G,A,1,,-5.644000,8.108000,-5.264000,0,0,O,
4,ATOM,4,O5',,G,A,1,,-6.900000,9.273000,-3.418000,0,0,O,
5,ATOM,5,HO5',,G,A,1,,-6.282181,8.322225,-3.067587,0,0,H,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,ATOM,323,H42,,C,A,10,,11.222122,3.811869,4.483277,0,0,H,
324,ATOM,324,C5,,C,A,10,,13.735000,4.259000,2.159000,0,0,C,
325,ATOM,325,H5,,C,A,10,,14.337950,3.264931,2.406216,0,0,H,


In [89]:
Coarse = CoarseGrain2(all_a,rna=True)
Coarse

,recname,serial,name,altLoc,resname,chainID,resSeq,iCode,x,y,z,occupancy,tempFactor,element,charge,type
0,ATOM,0,G,,G,A,1,,-10.918892,4.978591,-2.257049,0,0,C,,G
1,ATOM,1,Sr,,G,A,1,,-9.095890,9.292894,-1.229304,0,0,D,,S
2,ATOM,2,G,,G,A,2,,-9.414671,2.374248,0.401136,0,0,C,,G
3,ATOM,3,P,,G,A,2,,-5.978304,8.548315,0.721341,0,0,P,,P
4,ATOM,4,Sr,,G,A,2,,-8.666163,5.977498,3.475239,0,0,D,,S
5,ATOM,5,G,,G,A,3,,-7.121323,-0.703327,1.575372,0,0,C,,G
6,ATOM,6,P,,G,A,3,,-5.332261,5.211698,5.016760,0,0,P,,P
7,ATOM,7,Sr,,G,A,3,,-7.087119,1.052939,6.036926,0,0,D,,S
8,ATOM,8,G,,G,A,4,,-4.051170,-3.270847,1.270904,0,0,C,,G
9,ATOM,9,P,,G,A,4,,-3.567816,0.519672,7.223229,0,0,P,,P


In [90]:
writePDB(Coarse,'G5C5_clean.pdb')

In [117]:
import simtk.openmm
pdb=simtk.openmm.app.PDBFile('GlCl_clean.pdb')
top=pdb.topology
coord=pdb.positions
forcefield=simtk.openmm.app.ForceField('3SPN2.xml')
s=forcefield.createSystem(top)

#### Defining different types of interactions

In [118]:
#### bonds b/w atom defining

import MDAnalysis as mda
u = mda.Universe('GlCl_clean.pdb')

bond_sb =[]
bond_sp = []

for i in np.unique(u.atoms.chainIDs):
    for j in np.unique(u.select_atoms(f'chainID {i}').resids):
        
        sugar = u.select_atoms(f'chainID {i} and (resid {j} and (name Sr))').indices[0]
        base = u.select_atoms(f'chainID {i} and (resid {j} and (name G or name U or name A or name C))').indices[0]
        bond_sb.append((sugar,base))
        
        if j<len(np.unique(u.select_atoms(f'chainID {i}').resids)):
            pho_next = u.select_atoms(f'chainID {i} and (resid {j+1} and (name P))').indices[0]
            bond_sp.append((sugar,pho_next))


        if j>1:
            pho = u.select_atoms(f'chainID {i} and (resid {j} and (name P))').indices[0]
            bond_sp.append((pho,sugar))
            
bond_index = [bond_sb,bond_sp]

In [121]:
### Angle between atoms

angle_psg = []
angle_psp = []
angle_bsp = []
angle_sps = []

for i in np.unique(u.atoms.chainIDs):
    for j in np.unique(u.select_atoms(f'chainID {i}').resids):

        if j<len(np.unique(u.select_atoms(f'chainID {i}').resids)):
            
            sugar = u.select_atoms(f'chainID {i} and (resid {j} and (name Sr))').indices[0]
            base = u.select_atoms(f'chainID {i} and (resid {j} and (name G or name U or name A or name C))').indices[0]      
            pho_next = u.select_atoms(f'chainID {i} and (resid {j+1} and (name P))').indices[0]
            sugar_next = u.select_atoms(f'chainID {i} and (resid {j+1} and (name Sr))').indices[0]
            
            angle_bsp.append((base,sugar,pho_next))
            angle_sps.append((sugar,pho_next,sugar_next))
            
        if j>1:
            
            pho = u.select_atoms(f'chainID {i} and (resid {j} and (name P))').indices[0]            
            angle_psg.append((pho,sugar,base)) 
            
            
        if j>1 and j<len(np.unique(u.select_atoms(f'chainID {i}').resids)):
            
            angle_psp.append((pho,sugar,pho_next)) 
            
angles = [angle_psg,angle_psp,angle_bsp,angle_sps]

In [123]:
## Add dihedral angle
di_psps = []
di_spsp = []

for i in np.unique(u.atoms.chainIDs):
    for j in np.unique(u.select_atoms(f'chainID {i}').resids):
            
            if j < len(np.unique(u.select_atoms(f'chainID {i}').resids))-1:
                
                sugar = u.select_atoms(f'chainID {i} and (resid {j} and (name Sr))').indices[0]
                pho_next = u.select_atoms(f'chainID {i} and (resid {j+1} and (name P))').indices[0]
                s_next = u.select_atoms(f'chainID {i} and (resid {j+1} and (name Sr))').indices[0]
                pho_2next = u.select_atoms(f'chainID {i} and (resid {j+2} and (name P))').indices[0]
                
                di_spsp.append((sugar,pho_next,s_next,pho_2next))                
            
            if j>1 and j<len(np.unique(u.select_atoms(f'chainID {i}').resids)):
            
                pho = u.select_atoms(f'chainID {i} and (resid {j} and (name P))').indices[0]
                sugar = u.select_atoms(f'chainID {i} and (resid {j} and (name Sr))').indices[0]
                pho_next = u.select_atoms(f'chainID {i} and (resid {j+1} and (name P))').indices[0]
                s_next = u.select_atoms(f'chainID {i} and (resid {j+1} and (name Sr))').indices[0]
            
                di_psps.append((pho,sugar,pho_next,s_next))
            
            
dihedral = [di_psps,di_spsp]
print(len(di_psps),len(di_spsp))

45 45


In [124]:
### finding a hydrogen bond donor and acceptor

import MDAnalysis as mda
u = mda.Universe('GlCl_clean.pdb')

list_G = []  ### Take G as donor 
list_C = []  ### Take C as acceptor
list_A = []
list_U = []

for i in np.unique(u.atoms.chainIDs):
    for j in np.unique(u.select_atoms(f'chainID {i}').resids):
        
        if j<len(np.unique(u.select_atoms(f'chainID {i}').resids)):
            
            sugar = u.select_atoms(f'chainID {i} and (resid {j} and (name Sr))').indices[0]
            pho_next = u.select_atoms(f'chainID {i} and (resid {j+1} and (name P))').indices[0]
            base = u.select_atoms(f'chainID {i} and (resid {j} and (name G or name U or name A or name C))')
            
            if base.atoms[0].name =='G':
                list_G.append((base.indices[0],sugar,pho_next,j))
                
            if base.atoms[0].name =='C':
            
                list_C.append((base.indices[0],sugar,pho_next,j))
                
            if base.atoms[0].name =='A':
                list_A.append((base.indices[0],sugar,pho_next,j))
                
            if base.atoms[0].name =='U':
                list_U.append((base.indices[0],sugar,pho_next,j))

In [125]:
### Stacking interaction 

s_gg = []
s_gc = []
s_cc = []
s_cg = []
for i in np.unique(u.atoms.chainIDs):
    for j in np.unique(u.select_atoms(f'chainID {i}').resids):
        
        if j>1 and j<len(np.unique(u.select_atoms(f'chainID {i}').resids))-1:
            
            pho = u.select_atoms(f'chainID {i} and (resid {j} and (name P))').indices[0]
            sugar = u.select_atoms(f'chainID {i} and (resid {j} and (name Sr))').indices[0]
            pho_next = u.select_atoms(f'chainID {i} and (resid {j+1} and (name P))').indices[0]
            s_next = u.select_atoms(f'chainID {i} and (resid {j+1} and (name Sr))').indices[0]
            pho_2next = u.select_atoms(f'chainID {i} and (resid {j+2} and (name P))').indices[0]
            
            base = u.select_atoms(f'chainID {i} and (resid {j} and (name G or name U or name A or name C))')
            base_next = u.select_atoms(f'chainID {i} and (resid {j+1} and (name G or name U or name A or name C))')
            
            if base.atoms[0].name =='G'and base_next.atoms[0].name=='G':
                s_gg.append(('GG',base.indices[0],base_next.indices[0],pho,sugar,pho_next,s_next,pho_2next))
                
            if base.atoms[0].name =='G'and base_next.atoms[0].name=='C':
                s_gc.append(('GC',base.indices[0],base_next.indices[0],pho,sugar,pho_next,s_next,pho_2next))
                
            if base.atoms[0].name =='C'and base_next.atoms[0].name=='C':
                s_cc.append(('CC',base.indices[0],base_next.indices[0],pho,sugar,pho_next,s_next,pho_2next))
                
            if base.atoms[0].name =='C'and base_next.atoms[0].name=='G':
                s_cg.append(('CG',base.indices[0],base_next.indices[0],pho,sugar,pho_next,s_next,pho_2next))
                
stacking_all = [s_gg,s_gc,s_cc,s_cg]

In [126]:
####### Stacking interaction via lj
# stacking_lj = []
# for i in np.unique(u.atoms.chainIDs):
#     for j in np.unique(u.select_atoms(f'chainID {i}').resids):
        
#         if  j<len(np.unique(u.select_atoms(f'chainID {i}').resids)):
            
#             base = u.select_atoms(f'chainID {i} and (resid {j} and (name G or name U or name A or name C))')
#             base_next = u.select_atoms(f'chainID {i} and (resid {j+1} and (name G or name U or name A or name C))')            
#             stacking_lj.append((base.indices[0],base_next.indices[0]))

#### Defining parameters

In [128]:
##angle
PSP = np.deg2rad(82.735)
SPS = np.deg2rad(87.410)
PSA  = np.deg2rad(97.569)
PSU = np.deg2rad(90.155)
PSG = np.deg2rad(101.356)
PSC = np.deg2rad(90.545)
ASP = np.deg2rad(110.346)
USP = np.deg2rad(112.661)
GSP = np.deg2rad(109.721)
CSP = np.deg2rad(112.615)

##Bond
SP = 3.8157
PS = 4.6010
SA = 4.8515
SU = 4.2733
SG = 4.9659
SC = 4.2738

##dihedral angle
PSPS = np.deg2rad(-148.215)
SPSP = np.deg2rad(175.975)


###H-bond distance
A_U = 5.8815
G_C = 5.6550

###
SA_U = np.deg2rad(156.320)
SU_A = np.deg2rad(143.910)
SG_C = np.deg2rad(161.746)
SC_G = np.deg2rad(142.306)


###H-bond dihedral
SA_US = 71.958
SG_CS = 79.653
PSA_U = 54.689
PSU_A = 67.305
PSG_C = 43.654
PSC_G = 69.752

####stack distance
AC = 3.8260185
AG = 4.4255305
AU  = 3.8260185
CA  = 4.7010580
CC  = 4.2500910
CG  = 4.9790760
CU  = 4.2273615
GA  = 4.0128560
GC  = 3.6784360
GG  = 4.2427250
GU  = 3.6616930
UA  = 4.7010580
UC  = 4.2679180
UG  = 4.9977560
UU  = 4.2453650

###   h       s      Tm
s_AA = [4.348,  -0.319,   298.9]
s_AC = [4.311,  -0.319,   298.9]
s_AG = [5.116,   5.301,   341.2]
s_AU  =[4.311,  -0.319,   298.9]
s_CA  = [4.287,  -0.319,   298.9]
s_CC  = [4.015,  -1.567,   285.8]
s_CG  = [4.602,   0.774,   315.5]
s_CU  = [3.995,  -1.567,   285.8]
s_GA  = [5.079,   5.301,   341.2]
s_GC  = [5.075,   4.370,   343.2]
s_GG  = [5.555,   7.346,   366.3]
s_GU  = [4.977,   2.924 ,  338.2]
s_UA  = [4.287,  -0.319,   298.9]
s_UC  = [3.992,  -1.567,   285.8]
s_UG  = [5.032,   2.924,   338.2]
s_UU  = [3.370,  -3.563,   251.6]

kB=1.380649*10**(-23)
def s_u0(x,T):
    return -x[0] + kB*(T - x[2])* x[1]

In [129]:
ss_gg = s_u0(s_GG,300)
ss_gc = s_u0(s_GC,300)
ss_cg = s_u0(s_CG,300)
ss_cc = s_u0(s_CC,300)

all_ss = {'GG':(GG,ss_gg),'GC':(GC,ss_gc),'CC':(CC,ss_cc),'CG':(CG,ss_cg)} 

#### Adding interaction into system

In [131]:
##angle_psg,angle_psp,angle_bsp,angle_sps

In [132]:
import openmm as mm
from   openmm import app
from   openmm.unit import * 
from   openmm import unit
import parmed
import mdtraj as md
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

########## bond force
bondforce = mm.HarmonicBondForce()
b_value=[SG,SP]

for bond,value in zip(bond_index,b_value):
    for bond_ind in bond:
        bondforce.addBond(bond_ind[0], bond_ind[1], value*unit.angstroms,
                      20.0*unit.kilocalorie_per_mole/(unit.angstrom**2))

bondforce.setUsesPeriodicBoundaryConditions(True)
bondforce.setForceGroup(0)
s.addForce(bondforce)


### Angle
force_angle = mm.HarmonicAngleForce()
force_angle.setUsesPeriodicBoundaryConditions(True)
ang_values = [PSG,PSP,GSP,SPS]   ####[angle_psg,angle_psp,angle_bsp,angle_sps]
ang_k = 5.0*kilocalorie_per_mole/(radians**2)
for ang,values in zip(angles,ang_values):
    for ind_ang in ang:
        force_angle.addAngle(ind_ang[0], ind_ang[1], ind_ang[2],
                             values*radians, ang_k)
        
force_angle.setForceGroup(1)  
s.addForce(force_angle)

### Dihedral
force_dihedral = mm.PeriodicTorsionForce()
force_dihedral.setUsesPeriodicBoundaryConditions(True)

##dihedral angle
PSPS = np.deg2rad(-148.215)
SPSP = np.deg2rad(175.975)

di_values = [PSPS,SPSP]

di_k = 40.0*kilocalorie_per_mole/(radians**2)

for di, values in zip(dihedral,di_values):
    for ind_di in di:
        force_dihedral.addTorsion(ind_di[0],ind_di[1],ind_di[2],ind_di[3],1,
                                 values*radians,10*kilojoules_per_mole)

force_dihedral.setForceGroup(3)      
s.addForce(force_dihedral)

3

In [133]:
#### Hydrogen bond adding
import openmm.unit as u

Hbondingforce = mm.CustomCompoundBondForce(6,"scale1*UHyd/(1.0 + kdist*DIST*DIST + kang*ANG1*ANG1 + kang*ANG2*ANG2 + kdih*DIHED1*DIHED1 + kdih*DIHED2*DIHED2 + kdih*DIHED3*DIHED3); scale1 = 0.25*(1.0 - ((angle(p2,p1,p4)-pi_cut)/(abs(angle(p2,p1,p4)-pi_cut))))*(1.0 - ((angle(p5,p4,p1)-pi_cut)/(abs(angle(p5,p4,p1)-pi_cut)))); DIST = (distance(p1,p4) - dist_0); ANG1 = (angle(p2,p1,p4) - angle_01); ANG2 = (angle(p5,p4,p1) - angle_02); DIHED1 = dihedral(p2,p1,p4,p5) - dihedral_01 + pi*(((pi - dihedral(p2,p1,p4,p5) + dihedral_01)/abs(pi - dihedral(p2,p1,p4,p5) + dihedral_01)) - ((pi + dihedral(p2,p1,p4,p5) - dihedral_01)/abs(pi + dihedral(p2,p1,p4,p5) - dihedral_01))) ;  DIHED2 = (dihedral(p6,p5,p4,p1) - dihedral_02 + pi*(((pi - dihedral(p6,p5,p4,p1) + dihedral_02)/abs(pi - dihedral(p6,p5,p4,p1) + dihedral_02)) - ((pi + dihedral(p6,p5,p4,p1) - dihedral_02)/abs(pi + dihedral(p6,p5,p4,p1) - dihedral_02)))); DIHED3 = (dihedral(p3,p2,p1,p4) - dihedral_03 + pi*(((pi - dihedral(p3,p2,p1,p4) + dihedral_03)/abs(pi - dihedral(p3,p2,p1,p4) + dihedral_03)) - ((pi + dihedral(p3,p2,p1,p4) - dihedral_03)/abs(pi + dihedral(p3,p2,p1,p4) - dihedral_03))));")

Hbondingforce.addPerBondParameter("UHyd")
Hbondingforce.addPerBondParameter("dist_0")
Hbondingforce.addPerBondParameter("angle_01")
Hbondingforce.addPerBondParameter("angle_02")
Hbondingforce.addPerBondParameter("dihedral_01")
Hbondingforce.addPerBondParameter("dihedral_02")
Hbondingforce.addPerBondParameter("dihedral_03")
Hbondingforce.addGlobalParameter('pi_cut',3.124139361*radians)
Hbondingforce.addGlobalParameter('kdist', 4.00/angstroms**2)
Hbondingforce.addGlobalParameter('kang', 1.50/radians**2)
Hbondingforce.addGlobalParameter('kdih', 0.15/radians**2)
Hbondingforce.addGlobalParameter('pi', np.pi)
Hbondingforce.setUsesPeriodicBoundaryConditions(True)


#### Hydrogen bond between A-U

for donor in list_A:
    for acceptor in list_U:
        res_no_1  = donor[3]
        res_no_2  = acceptor[3]  ### base.indices[0],sugar,pho_next,reidue_no
        if abs(res_no_1-res_no_2)>1:   ### to avoid neighbour one
            p1 = donor[0]
            p2 = donor[1]
            p3 = donor[2]
            p4 = acceptor[0]
            p5 = acceptor[1]
            p6 = acceptor[2]
            dist_0 = A_U
            angle_01 = SA_U
            angle_02 = SU_A
            dihedral_01 = SA_US 
            dihedral_02 = PSA_U
            dihedral_03 = PSU_A
            UHyd = 3
            group_add2 = [p1,p2,p3,p4,p5,p6]
            Hbondingforce.addBond(group_add2, [UHyd*u.kilocalories_per_mole,
                                               dist_0*u.angstroms, angle_01*u.radians, 
                                               angle_02*u.radians, dihedral_01*u.radians, 
                                               dihedral_02*u.radians, dihedral_03*u.radians])

####  Hydrogen bond between G-C
for donor in list_G:
    for acceptor in list_C:
        res_no_1  = donor[3]
        res_no_2  = acceptor[3]
        if abs(res_no_1-res_no_2)>1:   ### to avoid neighbour one
            p1 = donor[0]
            p2 = donor[1]
            p3 = donor[2]
            p4 = acceptor[0]
            p5 = acceptor[1]
            p6 = acceptor[2]
            dist_0 = G_C
            angle_01 = SG_C
            angle_02 = SC_G
            dihedral_01 = SG_CS
            dihedral_02 = PSG_C
            dihedral_03 = PSC_G
            UHyd = 3
            group_add2 = [p1,p2,p3,p4,p5,p6]
            Hbondingforce.addBond(group_add2, [UHyd*u.kilocalories_per_mole,
                                               dist_0*u.angstroms, angle_01*u.radians, 
                                               angle_02*u.radians, dihedral_01*u.radians, 
                                               dihedral_02*u.radians, dihedral_03*u.radians])
            
# Hbondingforce.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)            
# Hbondingforce.setCutoffDistance(10.0 * angstroms)       # set cutoff (truncation) distance at 3*sigma
Hbondingforce.setForceGroup(4)         
s.addForce(Hbondingforce)

4

In [134]:
###################~~~~~~~~Add stacking interactions~~~~~~~~###################     

Stackingforce = mm.CustomCompoundBondForce(7, "U0/(1.0 + kbond*(distance(p6,p7) - r0)^2  + kphi1*(dihedral(p1,p2,p3,p4) - phi10 + pi*(((pi - dihedral(p1,p2,p3,p4) + phi10)/abs(pi - dihedral(p1,p2,p3,p4) + phi10)) -  ((pi + dihedral(p1,p2,p3,p4) - phi10)/abs(pi + dihedral(p1,p2,p3,p4) - phi10)))  )^2  +  kphi2*(dihedral(p2,p3,p4,p5) - phi20 + pi*(((pi - dihedral(p2,p3,p4,p5) + phi20)/abs(pi - dihedral(p2,p3,p4,p5) + phi20)) -  ((pi + dihedral(p2,p3,p4,p5) - phi20)/abs(pi + dihedral(p2,p3,p4,p5) - phi20)))  )^2 )");
Stackingforce.addPerBondParameter("U0");
Stackingforce.addPerBondParameter("r0");
Stackingforce.addPerBondParameter("phi10");
Stackingforce.addPerBondParameter("phi20");
Stackingforce.setUsesPeriodicBoundaryConditions(True)
Stackingforce.addGlobalParameter('kbond', 1.45/u.angstroms**2)
Stackingforce.addGlobalParameter('kphi1', 3.0/u.radians**2)
Stackingforce.addGlobalParameter('kphi2', 3.0/u.radians**2)
Stackingforce.addGlobalParameter('pi', np.pi)
Stackingforce.setUsesPeriodicBoundaryConditions(True)

for i in stacking_all:
    for j in i:
        
        ### ('GG',base.indices[0],base_next.indices[0],pho,sugar,pho_next,s_next,pho_2next)
        
        r0 = all_ss[j[0]][0]
        U0 = all_ss[j[0]][1]
        phi10 = PSPS
        phi20 = SPSP
        p1 = j[3]
        p2 = j[4]
        p3 = j[5]
        p4 = j[6]
        p5 = j[7]
        p6 = j[1]
        p7 = j[2]
        
        group_add = [p1, p2, p3, p4, p5, p6, p7]
        Stackingforce.addBond(group_add, [U0*u.kilocalories_per_mole,r0*u.angstroms, phi10*u.radians, phi20*u.radians])
        
Stackingforce.setForceGroup(5)          
s.addForce(Stackingforce)

5

In [135]:
# #### Adding stacking interaction via lj

# sig = 3.0 * angstrom
# eps = 117.8 * kelvin * BOLTZMANN_CONSTANT_kB * AVOGADRO_CONSTANT_NA

# E_lj  = '4*eps*((sig/r)^12-(sig/r)^6); sig=0.5*(sig1+sig2); eps=sqrt(eps1*eps2)'

# force_lj = mm.CustomNonbondedForce(E_lj)
# force_lj.addPerParticleParameter('sig') 
# force_lj.addPerParticleParameter('eps')

# # Particles are assigned properties in the same order as they appear in the System object
# for i in range(no_of_atoms):
#     force_lj.addParticle([sig, eps])  

# for i in stacking_lj:
#     force_lj.addInteractionGroup([i[0]],[i[1]])


# # Set force cutoff parameters

# force_lj.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
# force_lj.setCutoffDistance(3.0 * sig)       # set cutoff (truncation) distance at 3*sigma

        
# force_lj.setForceGroup(5)      
# s.addForce(force_lj)

6

In [136]:
# ### Adding LJ force to others 
# ### Create LJ force

# sig = 3.0 * angstrom
# eps = 117.8 * kelvin * BOLTZMANN_CONSTANT_kB * AVOGADRO_CONSTANT_NA

# E_lj  = '4*eps*((sig/r)^12-(sig/r)^6); sig=0.5*(sig1+sig2); eps=sqrt(eps1*eps2)'

# force_lj = mm.CustomNonbondedForce(E_lj)
# force_lj.addPerParticleParameter('sig') 
# force_lj.addPerParticleParameter('eps')

# # Particles are assigned properties in the same order as they appear in the System object
# u = mda.Universe('clean.pdb')
# natom = len(u.atoms)

# for _ in range(natom): 
#     force_lj.addParticle([sig, eps])
    
# # exlcude bonded particle
# for bond in bond_index:
#     for bond_ind in bond:     
#         force_lj.createExclusionsFromBonds([bond_ind], 1) 

# # Set force cutoff parameters

# force_lj.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
# force_lj.setCutoffDistance(3.0 * sig)       # set cutoff (truncation) distance at 3*sigma

        
# force_lj.setForceGroup(4)      
# s.addForce(force_lj)

#### Running a Simulation 

In [137]:
import numpy as np
temperature=300 * simtk.openmm.unit.kelvin

platform_name='OpenCL'

integrator = simtk.openmm.LangevinIntegrator(temperature, 1 / simtk.openmm.unit.picosecond,
                                             2 * simtk.openmm.unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)
simulation = simtk.openmm.app.Simulation(top,s, integrator, platform)
simulation.context.setPositions(coord)
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print(energy)

3089.5927734375


In [138]:
#Add simulation reporters
import sys
dcd_reporter=simtk.openmm.app.DCDReporter(f'output_lj3.dcd', 1000)

energy_reporter=simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,
                                                   potentialEnergy=True, temperature=True)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(energy_reporter)
simulation.reporters.append(app.PDBReporter('test.pdb', 100))

In [139]:
#Run simulation
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(100000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)"
1000,2.0000000000000013,1373.69189453125,247.5364293763014
2000,3.999999999999781,1314.9246826171875,283.2961000426657
3000,5.999999999999561,1395.8126220703125,297.02151955622554
4000,7.999999999999341,1357.343505859375,307.8442533778716
5000,10.000000000000009,1273.54150390625,298.82279309789595
6000,12.000000000000677,1179.0003662109375,262.68650705347557
7000,14.000000000001345,1075.786376953125,285.23139086623706
8000,16.00000000000201,998.7301635742188,324.08166726428567
9000,18.000000000000902,1055.96533203125,308.1672671071573
10000,19.999999999999794,946.1072998046875,281.59659685796674
11000,21.999999999998685,850.2708129882812,337.6709248842115
12000,23.999999999997577,852.2216186523438,308.63503614583567
13000,25.99999999999647,838.817138671875,328.52393852265243
14000,27.99999999999536,828.5457763671875,268.8789120145252
15000,29.99999999999425,718.9234008789062,336.0106193679377
16000,31.999999999993143,66

In [3]:
import nglview as nv
import mdtraj as md
u = md.load('output_lj2.dcd',top='GlCl_clean.pdb')
view = nv.show_mdtraj(u)
view.add_representation('ball+stick',color='red')

view.add_unitcell(color='white')
view

NGLWidget(max_frame=99)

In [40]:
forcefield    = ForceField('amber99sbildn.xml', 'tip4pew.xmlimport open3SPN2')

### Lj polymer 

In [28]:
import openmm as mm
from   openmm import app
from   openmm.unit import * 

import parmed
import mdtraj as md
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [33]:
from openmm.app import *
from openmm import *
from openmm.unit import *

In [31]:
import mbuild as mb

In [18]:
# Simulation parameters
temperature = 293.15 * kelvin
pressure = 1 * bar
mass = 39.948 * amu
sig = 3.419 * angstrom
eps = 117.8 * kelvin * BOLTZMANN_CONSTANT_kB * AVOGADRO_CONSTANT_NA

box_size = 150 * angstrom  
natom    = 12
cutoff   = 3 * sig

In [19]:
from parmed import Structure, Atom, Bond, Angle, Dihedral

s  = Structure()

for i in range(natom):
  s.add_atom(atom     =  Atom(name='A',  mass=mass), 
             resname  = "LJ", 
             resnum   =  i, 
             chain    = 'A')
  
s.positions = [(0, 0, z) for z in range(natom)]

for i in range(natom-1):
  s.bonds.append(Bond(s.atoms[i], s.atoms[i+1]))

s.save('lj.pdb', overwrite=True)
s.save('lj.psf', overwrite=True)

print(s.topology)
s.to_dataframe().head()

<Topology; 1 chains, 12 residues, 12 atoms, 11 bonds>


,number,name,type,atomic_number,charge,mass,nb_idx,solvent_radius,screen,occupancy,...,rmin_14,epsilon_14,resname,resid,resnum,chain,segid,xx,xy,xz
0,-1,A,,0,0.0,39.948,0,0.0,0.0,0.0,...,0.0,0.0,LJ,0,0,A,,0,0,0
1,-1,A,,0,0.0,39.948,0,0.0,0.0,0.0,...,0.0,0.0,LJ,1,1,A,,0,0,1
2,-1,A,,0,0.0,39.948,0,0.0,0.0,0.0,...,0.0,0.0,LJ,2,2,A,,0,0,2
3,-1,A,,0,0.0,39.948,0,0.0,0.0,0.0,...,0.0,0.0,LJ,3,3,A,,0,0,3
4,-1,A,,0,0.0,39.948,0,0.0,0.0,0.0,...,0.0,0.0,LJ,4,4,A,,0,0,4


In [20]:
# Add particles to the system
system = mm.System()

for i in range(natom):
  system.addParticle(s.atoms[i].mass)

box_vecs = box_size  *np.eye(3)
system.setDefaultPeriodicBoxVectors(*box_vecs )

In [21]:
### Create LJ force
E_lj  = '4*eps*((sig/r)^12-(sig/r)^6); sig=0.5*(sig1+sig2); eps=sqrt(eps1*eps2)'
force = mm.CustomNonbondedForce(E_lj)
force.addPerParticleParameter('sig') 
force.addPerParticleParameter('eps')

# Particles are assigned properties in the same order as they appear in the System object
for _ in range(natom): 
    force.addParticle([sig, eps])

# Set force cutoff parameters
force.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
force.setCutoffDistance(3.0 * sig)       # set cutoff (truncation) distance at 3*sigma
force.createExclusionsFromBonds([(i, i+1) for i in range(natom-1)], 1) # exlcude bonded particles from LJ

### Create Harmonic force
force2 = mm.HarmonicBondForce()

for i in range(natom-1):
  force2.addBond(i, i+1, 1.5*sig, 100) 

### Add a force to remove Center of Mass motion to prevent drifting of molecule
force3 = mm.CMMotionRemover()

# Added forces to system
system.addForce(force)
system.addForce(force2)
system.addForce(force3)

2

In [22]:
print('Check if force uses PBC: ', force.usesPeriodicBoundaryConditions() )

print('No particles: ', force.getNumParticles() )

print('per-particle parameters for particle-0: ', force.getParticleParameters(0))

print('Check Energy function: ', force.getEnergyFunction() )

Check if force uses PBC:  True
No particles:  12
per-particle parameters for particle-0:  (0.34190000000000004, 0.9794436964184517)
Check Energy function:  4*eps*((sig/r)^12-(sig/r)^6); sig=0.5*(sig1+sig2); eps=sqrt(eps1*eps2)


In [23]:
integrator =  mm.LangevinIntegrator(temperature, 1/picosecond, 2*femtoseconds) 

simulation = app.Simulation(s.topology, 
                            system, 
                            integrator) 

simulation.context.setPositions(s.positions)

# - Minimize the energy
simulation.minimizeEnergy()

# - Initialize velocities with random values at 300K.
simulation.context.setVelocitiesToTemperature(300)


# Reporters
simulation.reporters = []
simulation.reporters.append(app.DCDReporter('ljtraj.dcd', 1000))
simulation.reporters.append(app.PDBReporter('ljtraj.pdb', 1000))
simulation.reporters.append(app.StateDataReporter("ljscalars.csv", 10, 
                                              time=True, 
                                              potentialEnergy=True, 
                                              totalEnergy=True, 
                                              temperature=True, 
                                              volume=True))


In [24]:
simulation.step(100000)

In [5]:
import nglview as nv
import mdtraj as md

In [6]:
u = md.load('ljtraj.dcd',top='ljtraj.pdb')
view = nv.show_mdtraj(u)
view.add_representation('ball+stick',color='red')
view.add_unitcell(color='white')
view

NGLWidget(max_frame=99)

### Diherdral from struture 

In [3]:
import MDAnalysis as mda
from MDAnalysis.analysis.dihedrals import Dihedral

In [112]:
u = mda.Universe('GlCl_clean.pdb','output_lj3.dcd')

/opt/homebrew/anaconda3/envs/openmm_all/lib/python3.9/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [113]:
j=3
sugar = u.select_atoms(f'(resid {j} and (name Sr))')
pho_next = u.select_atoms(f'(resid {j+1} and (name P))')
s_next = u.select_atoms(f'(resid {j+1} and (name Sr))')
pho_2next = u.select_atoms(f'(resid {j+2} and (name P))')


##Dihedral([[sugar,pho_next,s_next,pho_2next]])

In [114]:
import numpy as np
import MDAnalysis as mda

u.trajectory[60]
# Define the atom selections
j = 1
sugar = u.select_atoms(f"(resid {j} and name Sr)")
pho_next = u.select_atoms(f"(resid {j+1} and name P)")
s_next = u.select_atoms(f"(resid {j+1} and name Sr)")
pho_2next = u.select_atoms(f"(resid {j+2} and name P)")


In [115]:
import numpy as np

def calc_dihedral(u1, u2, u3, u4):
    """ Calculate dihedral angle method. From bioPython.PDB
    (adapted to np.array)
    Calculate the dihedral angle between 4 vectors
    representing 4 connected points. The angle is in
    [-pi, pi].
    """

    a1 = u2 - u1
    a2 = u3 - u2
    a3 = u4 - u3

    v1 = np.cross(a1, a2)
    v1 = v1 / (v1 * v1).sum(-1)**0.5
    v2 = np.cross(a2, a3)
    v2 = v2 / (v2 * v2).sum(-1)**0.5
    porm = np.sign((v1 * a3).sum(-1))
    rad = np.arccos((v1*v2).sum(-1) / ((v1**2).sum(-1) * (v2**2).sum(-1))**0.5)
    if not porm == 0:
        rad = rad * porm

    return rad

In [116]:
calc_dihedral(sugar.positions[0], pho_next.positions[0],s_next.positions[0],pho_2next.positions[0])

-0.3210944672561437

In [44]:

j=4

pho = u.select_atoms(f'(resid {j} and (name P))').positions[0]
sugar = u.select_atoms(f'(resid {j} and (name Sr))').positions[0]
pho_next = u.select_atoms(f'(resid {j+1} and (name P))').positions[0]
s_next = u.select_atoms(f'(resid {j+1} and (name Sr))').positions[0]


calc_dihedral(pho,sugar,pho_next,s_next)

-2.592801470421043